<a href="https://colab.research.google.com/github/koushikpmv/Jio-anaemia-detection-website-data-/blob/main/Anaemia_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import glob
from PIL import Image
from tensorflow import keras
from keras import models, layers
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import rotate
from scipy import ndimage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
def img_to_numpy_array(folder_path):   #functioin to return numpy array of all the images in the specified folder
  data_path = os.path.join(folder_path,'*g')
  files = glob.glob(data_path)
  img_arr = []
  label_arr = []
  if folder_path == anaemic_folder:
    for image in files:
      img = cv2.imread(image)
      img_arr.append(img)
      label_arr.append(1)
  else:
    for image in files:
      img = cv2.imread(image)
      img_arr.append(img)
      label_arr.append(0)
  img_arr = np.array(img_arr)
  label_arr = np.array(label_arr)
  img_arr =  img_arr/255
  return img_arr,label_arr        

In [97]:
def mean_intensity_array(data_arr):                   #return array containing mean intensitites of all the images in the given array
  def mean_intensity_single_image(image,color):       #return mean intensity of a color (red,green,blue) of a single image
    rows,columns,rgb = image.shape                    #color in {0,1,2} for R,G,B
    intensity = 0
    for i in range(rows):
      for j in range(columns):
        intensity = intensity + image[i][j][color]
    mean_intensity_ = intensity/(rows*columns)
    return mean_intensity_
  mean_intensity_arr=[]
  for img in data_arr:
    mean_intensity_arr.append([mean_intensity_single_image(img,0),mean_intensity_single_image(img,1)])
  mean_intensity_arr = np.array(mean_intensity_arr)
  return mean_intensity_arr

In [ ]:
def preprocessing_crop_conjunctiva(image):
  pass

In [ ]:
def img_augmentation():                                                   #Getting error while implementing
  data_gen = ImageDataGenerator(rotation_range=45,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode='constant')
  return data_gen

In [94]:
def img_augment(x_train,y_train):                                 #Gives augemented (image, label arrays) of 4 times the number of input images.
  train_x = x_train
  train_y = y_train
  final_train_data = []
  final_target_train = []
  for i in range(train_x.shape[0]):
      final_train_data.append(train_x[i])
      final_train_data.append(rotate(train_x[i], angle=45, mode = 'wrap'))
      final_train_data.append(np.fliplr(train_x[i]))
      final_train_data.append(np.flipud(train_x[i]))
      for j in range(4):
          final_target_train.append(train_y[i])
  final_train_data = np.array(final_train_data)
  final_target_train = np.array(final_target_train)
  return final_train_data,final_target_train

In [ ]:
img_data_folder = '/content/drive/MyDrive/Detection_anaemia'
anaemic_folder = '/content/drive/MyDrive/Detection_anaemia/Anaemic'
non_anaemic_folder = '/content/drive/MyDrive/Detection_anaemia/Non-Anaemic'

In [ ]:
anaemic_arr,label_anaemic = img_to_numpy_array(anaemic_folder)

In [ ]:
non_anaemic_arr,label_non_anaemic = img_to_numpy_array(non_anaemic_folder)

In [ ]:
data_arr = np.concatenate((anaemic_arr,non_anaemic_arr))
label_arr = np.concatenate((label_anaemic,label_non_anaemic))

In [ ]:
x_train,x_test, y_train,y_test = train_test_split(data_arr,label_arr,test_size=0.2)

In [ ]:
(x_train.shape,x_test.shape),(y_train.shape,y_test.shape)

(((146, 224, 224, 3), (37, 224, 224, 3)), ((146,), (37,)))

In [95]:
final_data_arr,final_label_arr = img_augment(x_train,y_train)

In [98]:
final_data_arr.shape                #Number of images after augmentation = 584

(584, 224, 224, 3)

In [99]:
cnn = models.Sequential([
                         
                         layers.Conv2D(32,3,activation='relu',input_shape = (224,224,3)),
                         layers.MaxPooling2D(),
                         layers.BatchNormalization(),

                         layers.Conv2D(64,3,activation='relu'),
                         layers.MaxPooling2D(),
                         layers.BatchNormalization(),

                         layers.Flatten(),
                         layers.Dense(128,activation='relu'),
                         layers.BatchNormalization(),
                         layers.Dense(128,activation='relu'),
                         layers.BatchNormalization(),

                         layers.Dense(1,activation='sigmoid')

])

In [ ]:
cnn.compile(optimizer='adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

In [ ]:
cnn.fit(final_data_arr,final_label_arr,epochs=10)

Epoch 1/10
19/19 [==============================] - 56s 2s/step - loss: 1.6673 - accuracy: 0.9366
Epoch 2/10
19/19 [==============================] - 32s 2s/step - loss: 0.0437 - accuracy: 0.9932
Epoch 3/10
19/19 [==============================] - 32s 2s/step - loss: 0.0047 - accuracy: 1.0000
Epoch 4/10
19/19 [==============================] - 32s 2s/step - loss: 0.0681 - accuracy: 0.9932
Epoch 5/10
19/19 [==============================] - 32s 2s/step - loss: 0.3799 - accuracy: 0.9812
Epoch 6/10
19/19 [==============================] - 32s 2s/step - loss: 0.1299 - accuracy: 0.9897
Epoch 7/10
19/19 [==============================] - 32s 2s/step - loss: 0.0119 - accuracy: 0.9983
Epoch 8/10
19/19 [==============================] - 33s 2s/step - loss: 1.3207e-06 - accuracy: 1.0000
Epoch 9/10
19/19 [==============================] - 36s 2s/step - loss: 0.0089 - accuracy: 0.9983
Epoch 10/10
19/19 [==============================] - 32s 2s/step - loss: 0.0303 - accuracy: 0.9949


In [ ]:
cnn.evaluate(x_test,y_test)

2/2 [==============================] - 1s 84ms/step - loss: 7.4250 - accuracy: 0.7027


[7.42500114440918, 0.7027027010917664]